In [1]:
from openslrdata import load_dataset
from datasets import Dataset as HFdataset

# Load the dataset

dataset = load_dataset("./openslr/")

In [2]:
# do a train test split
dataset = dataset.train_test_split()

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file_path', 'folder_name', 'text'],
        num_rows: 164027
    })
    test: Dataset({
        features: ['file_path', 'folder_name', 'text'],
        num_rows: 54676
    })
})

In [4]:
# load whishperFeatureExtractor
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [5]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Bengali", task="transcribe")

In [6]:
input_str = dataset["train"][0]["text"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


2023-07-24 16:35:05.963000: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-24 16:35:06.005368: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Input:                 কমপক্ষে ২০টি গবেষণাকেন্দ্র
Decoded w/ special:    <|startoftranscript|><|bn|><|transcribe|><|notimestamps|>কমপক্ষে ২০টি গবেষণাকেন্দ্র<|endoftext|>
Decoded w/out special: কমপক্ষে ২০টি গবেষণাকেন্দ্র
Are equal:             True


In [7]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Bengali", task="transcribe")


In [8]:
from openslrdata import load_dataset
import librosa
from datasets import IterableDataset

In [9]:
def dataset_generator(dataset, feature_extractor, tokenizer, sampling_rate):
    for sample in dataset:
        audio_array, _ = librosa.load(sample["file_path"], sr=sampling_rate)
        yield {
            "input_features": feature_extractor(audio_array, sampling_rate=sampling_rate).input_features[0], 
            "labels": tokenizer(sample["text"]).input_ids
        }

def load_iterable_dataset(root_path, feature_extractor, tokenizer, sampling_rate):
    # Load the regular Hugging Face dataset
    dataset = load_dataset(root_path)

    # Convert the Hugging Face dataset into an iterable dataset
    iterable_dataset = IterableDataset(dataset_generator(dataset, feature_extractor, tokenizer, sampling_rate))

    return iterable_dataset

In [20]:
dataset = load_iterable_dataset("./openslr/", feature_extractor, tokenizer, sampling_rate=16000)

In [21]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [22]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [23]:
import evaluate

metric = evaluate.load("wer")


In [24]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [25]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


In [26]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [17]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-test",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)


AttributeError: 'IterableDataset' object has no attribute 'train_test_split'

In [18]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


In [19]:
trainer.train()

/home/venom/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/4000 [00:00<?, ?it/s]

/home/venom/miniconda3/lib/python3.9/site-packages/datasets/formatting/formatting.py:100: RuntimeWarning: divide by zero encountered in remainder
  return table.fast_gather(key % table.num_rows)


IndexError: list index out of range